# Setup

## Installing Necessary Packages 
Uncomment and Run If Required Packages aren't Installed

In [2]:
#import Pkg
#Pkg.add("QuantumOptics")
#Pkg.add("Logging")
#Pkg.add("LinearAlgebra")
#Pkg.add("SparseArrays")
#Pkg.add("ProtoStructs")
#Pkg.add("CSV")
#Pkg.add("MiniLoggers")

## Imports

In [3]:
import QuantumOptics as qo
using Logging

using LinearAlgebra
using SparseArrays
import CairoMakie as cm

using ProtoStructs

import QuantumOptics.⊗
import QuantumOptics.*

import CSV

using MiniLoggers

## Logger Setup

In [4]:
# MiniLogger(minlevel = MiniLoggers.Info) |> global_logger
InfoLogger = MiniLogger(minlevel = MiniLoggers.Info)
DebugLogger = MiniLogger(minlevel = MiniLoggers.Debug)

global_logger(InfoLogger)

Base.CoreLogging.SimpleLogger(VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(RawFD(21) open, 0 bytes waiting)), VSCodeServer.io_send_callback), Info, Dict{Any, Int64}())

## Random Functions

In [6]:
function tostr(obj)
    io = IOBuffer()
    show(io, "text/plain", obj)
    String(take!(io))
end

tostr (generic function with 1 method)

# Creating Structure and Initializer

## Definining Transmon_Resonators Struct

In [7]:
@proto @kwdef struct Transmon_Resonators
    Eᶜ :: Float64
    Eⱼ :: Float64
    gs :: Vector{Float64}
    ωs :: Vector{Float64}
    
    Nₜ :: Int
    Nₜ_trunc :: Int
    Nᵣs :: Vector{Int}
    Nᵣs_trunc :: Vector{Int}
    ng :: Real 
    
    𝔹 :: qo.NLevelBasis{Int64}
    Ĥ :: qo.Operator
    n̂ :: qo.Operator
    âs :: Vector

    dressed_states :: Dict
    dressed_energies :: Dict
    pulse_args :: Dict = Dict{Any, Any}()
    Extra_Stuf :: Dict = Dict{Any, Any}()
    dressed_states_overlap_dict :: Dict
end



## Initializer

In [11]:
function Make_Model(Eᶜ, Eⱼ, gs, ωs, Nₜ, Nᵣs; ng =0, Nₜ_trunc = 60, Nᵣs_trunc = 30, return_dressed_overlaps = false, sparse_eigs_factor = 2, state_match_factor = 1)
    # This function gets the truncated transmon hamiltonian and number operator in the energy basis 
    if !(length(ωs) == length(Nᵣs) & length(Nᵣs) == length(gs))
        @error "The lengths of g, ω and Nᵣs not the same. Please Try again"
        return Nothing
    end 
    Nᵣs_trunc = fill(Nᵣs_trunc, length(Nᵣs))

    # Building Non Truncated Transmon Ham
    # =================================================================================================
    @info "Building Non-Truncated Hamiltonians"
    𝔹ₜᶠᵘˡˡ = qo.NLevelBasis(2*Nₜ_trunc+1)
    𝕀̂ₜᶠᵘˡˡ=  qo.identityoperator(𝔹ₜᶠᵘˡˡ)
    jump = 0*𝕀̂ₜᶠᵘˡˡ
    for i in 1:(2*Nₜ_trunc)
        jump += qo.transition(𝔹ₜᶠᵘˡˡ, i, i+1)
    end

    n̂ₜᶠᵘˡˡ = 0*𝕀̂ₜᶠᵘˡˡ
    for i in 1:(2*Nₜ_trunc+1)
        n̂ₜᶠᵘˡˡ += qo.transition(𝔹ₜᶠᵘˡˡ, i, i)*(i-Nₜ_trunc-1)
    end

    Ĥₜᶠᵘˡˡ = 4Eᶜ*(ng*𝕀̂ₜᶠᵘˡˡ - n̂ₜᶠᵘˡˡ)^2 - 0.5*Eⱼ*(jump+jump')
    # =================================================================================================

    # Building Non-Truncated Resonator Hamiltonians
    # =================================================================================================
    @info "Building Non-Truncated Resonator Hamiltonians"
    𝔹ᵣᶠᵘˡˡs = []
    𝕀̂ᵣᶠᵘˡˡs = []
    âᵣᶠᵘˡˡs = []
    Ĥᵣᶠᵘˡˡs = []

    @debug "Building Resonator Hamiltonians"
    for i in 1:length(Nᵣs_trunc)
        @debug "On Resonator $i"
        Nᵣ = Nᵣs_trunc[i]
        𝔹ᵣ = qo.FockBasis(Nᵣ);
        push!(𝔹ᵣᶠᵘˡˡs, 𝔹ᵣ)
        âᵣ = qo.create(𝔹ᵣ)
        push!(âᵣᶠᵘˡˡs, âᵣ);
        push!(𝕀̂ᵣᶠᵘˡˡs, qo.identityoperator(𝔹ᵣ))
        push!(Ĥᵣᶠᵘˡˡs,  ωs[i]*âᵣ'*âᵣ)
    end 
    

    if length(Nᵣs_trunc) > 1
        Ĥᵣᶠᵘˡˡ = qo.tensor(𝕀̂ᵣᶠᵘˡˡs...)*0
        for i in 1:length(Nᵣs_trunc)
            ops = copy(𝕀̂ᵣᶠᵘˡˡs)
            ops[i] = Ĥᵣᶠᵘˡˡs[i]

            Ĥᵣᶠᵘˡˡ += qo.tensor(ops...)
        end
    end 
    if length(Nᵣs_trunc) == 1
        Ĥᵣᶠᵘˡˡ = Ĥᵣᶠᵘˡˡs[1]
    end
    # # =================================================================================================

    # # Building Non-Truncated Interaction Hamiltonian
    # # =================================================================================================
    @info "Building Non-Truncated Interaciton Hamiltonian"
    Ĥᵢᶠᵘˡˡ = (𝕀̂ₜᶠᵘˡˡ ⊗ Ĥᵣᶠᵘˡˡ)*0

    for i in 1:length(Nᵣs)
        @debug "Getting Interaction Hamiltonian part $i"
        ops = copy(𝕀̂ᵣᶠᵘˡˡs)
        @debug "    Got List of identity ops"
        ops[i] = âᵣᶠᵘˡˡs[i] + âᵣᶠᵘˡˡs[i]' 
        @debug "    Updated the $i op to âs + âs'"
        if length(ops)>1
            @debug "Length ops > 1"
            Ĥᵢᶠᵘˡˡ += n̂ₜᶠᵘˡˡ⊗qo.tensor(ops...)*gs[i]
        end
        if length(ops) == 1
            @debug "length ops == 1"
            Ĥᵢᶠᵘˡˡ += n̂ₜᶠᵘˡˡ⊗ops[1]*gs[i]
        end
        @debug "    Added the to the full interaction hamiltonian"
    end
    
    # # Getting Full Truncated Hamiltonian Hamiltonian
    # # =================================================================================================
    @info "Putting Together Full Hamiltonian"
    if length(𝕀̂ᵣᶠᵘˡˡs)>1
        @debug "Length 𝕀̂ᵣᶠᵘˡˡs>1"
        Ĥᶠᵘˡˡ = Ĥₜᶠᵘˡˡ⊗qo.tensor(𝕀̂ᵣᶠᵘˡˡs...)
        @debug "    Added Transmon"
        Ĥᶠᵘˡˡ += 𝕀̂ₜᶠᵘˡˡ⊗Ĥᵣᶠᵘˡˡ
        @debug "    Added Resonators"
        Ĥᶠᵘˡˡ+=  Ĥᵢᶠᵘˡˡ
        @debug "    Added Interactions"
    end 
    if length(𝕀̂ᵣᶠᵘˡˡs)==1
        Ĥᶠᵘˡˡ = Ĥₜᶠᵘˡˡ⊗𝕀̂ᵣᶠᵘˡˡs[1]+𝕀̂ₜᶠᵘˡˡ⊗Ĥᵣᶠᵘˡˡ + Ĥᵢᶠᵘˡˡ
    end 

    # # Labeling Dressed States
    # # =================================================================================================
    @info "Getting Eigenstates of Dressed and Bare Hamiltonians"
    FINAL_DIM = (Nₜ+1)*prod((Nᵣs .+ 1)) # Final DIM
    num_eigstates = trunc(Int, sparse_eigs_factor*(FINAL_DIM))
    @debug "    Getting Dressed Eigensystem (This is Sparse)"
    dressed_eigsys = qo.eigenstates(Ĥᶠᵘˡˡ, n = num_eigstates)
    
    num_eigstates = sparse_eigs_factor*(Nₜ)
    @debug "    Getting Bare Transmon Eigensystem (This is Dense)"
    bare_transmon_states = qo.eigenstates(qo.dense(Ĥₜᶠᵘˡˡ))
    

    @debug "    Getting Bare Resonator Eigensystems (This is Dense)"
    bare_resonator_states_vec = []
    for i in 1:length(Nᵣs)
        push!(bare_resonator_states_vec, qo.eigenstates(qo.dense(Ĥᵣᶠᵘˡˡs[i])))
    end 

    overlap_dict = Dict{Any, Any}()

    res_iter_list = []
    for Nᵣᵢ in Nᵣs
        push!(res_iter_list, 0:Nᵣᵢ)
    end

    states_to_keep_iter = Iterators.product(0:Nₜ, res_iter_list...)
    
    num_to_check = trunc(Int, state_match_factor * FINAL_DIM)
    
    @info "Finding and Labeling Dressed States"
    @debug "    Building Overlap Dict"
    for state in states_to_keep_iter
        @debug "        Doing State $state"
        overlap_dict[state] = zeros(length(dressed_eigsys[1]))
        ψᵇᵃʳᵉₜ = bare_transmon_states[2][state[1]+1]
        ψᵇᵃʳᵉᵣs = []
        for i in 1:length(Nᵣs)
            push!(ψᵇᵃʳᵉᵣs, bare_resonator_states_vec[i][2][state[i+1]+1])
        end

        ψᵇᵃʳᵉ = qo.tensor(ψᵇᵃʳᵉₜ, ψᵇᵃʳᵉᵣs...)

        for i in 1:length(dressed_eigsys[2])
            ψᵈʳᵉˢˢᵉᵈ = dressed_eigsys[2][i]
            overlap_dict[state][i] = abs(ψᵈʳᵉˢˢᵉᵈ'*ψᵇᵃʳᵉ)^2
        end
    end
    
    dressed_states_dict_full = Dict{Any, Any}()
    dressed_states_energy_dict = Dict{Any, Any}()
    @debug "    Appending Results to Dressed Dicts"
    for state in states_to_keep_iter
        state_idx = argmax(overlap_dict[state])
        dressed_states_dict_full[state] = dressed_eigsys[2][state_idx]
        dressed_states_energy_dict[state] = dressed_eigsys[1][state_idx]
    end

    
    # Building Truncated Hilbertspace and Operators
    # =================================================================================================
    @info "Building Truncated Hilbertspace and Operators"

    dressed_states_dict_trunc = Dict{Any, Any}()

    if length(Nᵣs) >1
        n̂ᶠᵘˡˡ = qo.tensor(n̂ₜᶠᵘˡˡ, 𝕀̂ᵣᶠᵘˡˡs...) 
    end
    if length(Nᵣs) == 1
        n̂ᶠᵘˡˡ = qo.tensor(n̂ₜᶠᵘˡˡ, 𝕀̂ᵣᶠᵘˡˡs[1]) 
    end
    

    @debug "Building Projection Op"
    Π = zeros(ComplexF64, length(dressed_states_dict_full), size(Ĥᶠᵘˡˡ)[1])

    dressed_states_dict_full_keys = collect(keys(dressed_states_dict_full))
    for i in 1:length(dressed_states_dict_full_keys)
        state = dressed_states_dict_full_keys[i]
        Π[i, :] = dressed_states_dict_full[state].data
    end


    n⃗ = Π*n̂ᶠᵘˡˡ.data*Π'
    H⃗ = Π*Ĥᶠᵘˡˡ.data*Π'



    a⃗s = []
    for i in 1:length(Nᵣs)
        ops = copy(𝕀̂ᵣᶠᵘˡˡs)
        ops[i] = âᵣᶠᵘˡˡs[i]

        âᶠᵘˡˡᵢ = qo.tensor(𝕀̂ₜᶠᵘˡˡ, ops...)
        push!(a⃗s, Π*âᶠᵘˡˡᵢ.data*Π')
    end

    𝔹 = qo.NLevelBasis(FINAL_DIM)
    Ψ = qo.nlevelstate(𝔹, 1)
    
    n̂ = 0*qo.identityoperator(𝔹)
    n̂.data = n⃗

    Ĥ = 0*qo.identityoperator(𝔹)
    Ĥ.data = H⃗

    herm_check = norm((Ĥ-Ĥ').data)
    if herm_check>1e-9
        @error "herm check failed with value $herm_check"
        return Nothing
    end
    Ĥ = 0.5*(Ĥ+Ĥ')

    âs = []
    for i in 1:length(Nᵣs)
        âᵢ = 0*qo.identityoperator(𝔹)
        âᵢ.data = a⃗s[i]
        push!(âs, âᵢ)
    end

    for i in 1:length(dressed_states_dict_full_keys)
        state = dressed_states_dict_full_keys[i]
        ψᶠᵘˡˡ = dressed_states_dict_full[state]
        ψ = copy(Ψ)
        ψ.data = Π*ψᶠᵘˡˡ.data
        
        dressed_states_dict_trunc[state] = ψ
    end


    dressed_states_overlap_dict = Dict{Any, Any}()
    if return_dressed_overlaps
        dressed_states_overlap_dict = overlap_dict
    end

    @info "Done!"    
    return Transmon_Resonators(Eᶜ = Eᶜ, Eⱼ = Eⱼ, gs = gs, ωs = ωs, Nₜ = Nₜ, Nᵣs = Nᵣs, Nₜ_trunc = Nₜ_trunc, Nᵣs_trunc = Nᵣs_trunc, ng = ng, 𝔹 = 𝔹, Ĥ = Ĥ, n̂ = n̂, âs = âs, dressed_states = dressed_states_dict_trunc, dressed_energies = dressed_states_energy_dict, dressed_states_overlap_dict = dressed_states_overlap_dict)

    
end

Make_Model (generic function with 1 method)

# Testing 

In [12]:
Eᶜ = 0.10283303447280807;
Eⱼ = 26.96976142643705;

gs = [0.026184375000000013];
ωs = [6.2280871213];
Nₜ = 10
Nᵣs = [10]


Mode3 = Make_Model(Eᶜ, Eⱼ, gs, ωs, Nₜ, Nᵣs, Nₜ_trunc = 40, Nᵣs_trunc = 10, sparse_eigs_factor = 2);


[2024-08-22 19:52:51] Info: Building Non-Truncated Hamiltonians
[2024-08-22 19:52:51] Info: Building Non-Truncated Resonator Hamiltonians
[2024-08-22 19:52:51] Info: Building Non-Truncated Interaciton Hamiltonian
[2024-08-22 19:52:51] Info: Putting Together Full Hamiltonian
[2024-08-22 19:52:51] Info: Getting Eigenstates of Dressed and Bare Hamiltonians
[2024-08-22 19:52:54] Info: Defaulting to sparse diagonalization for sparse operator. If storing the full operator is possible, it might be faster to do `eigenstates(dense(op))`. Set `info=false` to turn off this message.
[2024-08-22 19:52:55] Info: Finding and Labeling Dressed States
[2024-08-22 19:52:55] Info: Building Truncated Hilbertspace and Operators
[2024-08-22 19:52:55] Info: Done!


In [13]:
Eᶜ = 0.10283303447280807;
Eⱼ = 26.96976142643705;

gs = [0.026184375000000013, 0.03020625000000001];
ωs = [6.2280871213, 6.72062927579];
Nₜ = 10
Nᵣs = [6,6]


Mode35 = Make_Model(Eᶜ, Eⱼ, gs, ωs, Nₜ, Nᵣs, Nₜ_trunc = 40, Nᵣs_trunc = 10, sparse_eigs_factor = 2);


[2024-08-22 19:52:57] Info: Building Non-Truncated Hamiltonians
[2024-08-22 19:52:57] Info: Building Non-Truncated Resonator Hamiltonians
[2024-08-22 19:52:57] Info: Building Non-Truncated Interaciton Hamiltonian
[2024-08-22 19:52:57] Info: Putting Together Full Hamiltonian
[2024-08-22 19:52:57] Info: Getting Eigenstates of Dressed and Bare Hamiltonians
[2024-08-22 19:52:57] Info: Defaulting to sparse diagonalization for sparse operator. If storing the full operator is possible, it might be faster to do `eigenstates(dense(op))`. Set `info=false` to turn off this message.
[2024-08-22 19:55:57] Info: Finding and Labeling Dressed States
[2024-08-22 19:56:08] Info: Building Truncated Hilbertspace and Operators
[2024-08-22 19:56:10] Info: Done!


In [15]:
CSV.write("comparison_3.csv", Mode3.dressed_energies)
CSV.write("comparison_35.csv", Mode35.dressed_energies)

"comparison_35.csv"